In [1]:
# install openmodelica from its repository

shell_call <- function(command, ...) { # required for later call of shell...
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

shell_call('for deb in deb deb-src; do echo "$deb http://build.openmodelica.org/apt `lsb_release -cs` release"; done | sudo tee /etc/apt/sources.list.d/openmodelica.list')
shell_call('wget -q http://build.openmodelica.org/apt/openmodelica.asc -O- | sudo apt-key add -')
shell_call('apt update && apt install openmodelica')

deb http://build.openmodelica.org/apt bionic release
deb-src http://build.openmodelica.org/apt bionic releaseOKGet:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Ign:6 http://build.openmodelica.org/apt bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://build.openmodelica.org/apt bionic Release [26.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://build.o

In [2]:
# install Funz through devtools
devtools::install_github('Funz/Funz.R')
library(Funz)

iterators (NA -> 1.0.14) [CRAN]
foreach   (NA -> 1.5.2 ) [CRAN]
Rserve    (NA -> 1.8-10) [CRAN]
rJava     (NA -> 1.0-6 ) [CRAN]


Installing 4 packages: iterators, foreach, Rserve, rJava

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



* checking for file ‘/tmp/RtmpKMySID/remotes4a442571b8/Funz-Funz.R-77484ce/DESCRIPTION’ ... OK
* preparing ‘Funz’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘Funz_1.15-0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: rJava



In [3]:
# install plugin to support Modelica I/O
install.Model('Modelica', edit.script=FALSE)

...........

Installed Funz model Modelica



In [4]:
# setup main input file
writeLines(con='NewtonCooling.mo',text=c(
'// @ref http://book.xogeny.com/behavior/equations/physical/',
'model NewtonCooling "An example of Newton s law of cooling"',
'  parameter Real T_inf=25 "Ambient temperature";',
'  parameter Real T0=90 "Initial temperature";',
'  parameter Real h=0.7 "Convective cooling coefficient";',
'  parameter Real A=1.0 "Surface area";',
'  parameter Real m=0.1 "Mass of thermal capacitance";',
'  parameter Real c_p=1.2 "Specific heat";',
'  Real T "Temperature";',
'initial equation',
'  T = T0 "Specify initial value for T";',
'equation',
'  m*c_p*der(T) = h*A*(T_inf-T) "Newton s law of cooling";',
'end NewtonCooling;"'
))

In [5]:
# run this file with Funz (that will check everything is ok for later parametric study)
startCalculators(1)
Run(model="Modelica",input.files="NewtonCooling.mo")

[[1]]
PROCESS 'bash', running, pid 9982.


Using default input design.
Using default output expressions: T time
Using archive directory: /content
 Over.                                                                           

Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”


$time
$time[[1]]
[1] "[]"


$T
$T[[1]]
numeric(0)


$state
$state[[1]]
[1] "done"


$duration
$duration[[1]]
[1] 2


$calc
$calc[[1]]
[1] "localhost:41649"

In [6]:
# setup main parametrized (with `$convection`) input file
writeLines(con='NewtonCooling.mo.par',text=c(
'// @ref http://book.xogeny.com/behavior/equations/physical/',
'model NewtonCooling "An example of Newton s law of cooling"',
'  parameter Real T_inf=25 "Ambient temperature";',
'  parameter Real T0=90 "Initial temperature";',
'  parameter Real h=$convection "Convective cooling coefficient";',
'  parameter Real A=1.0 "Surface area";',
'  parameter Real m=0.1 "Mass of thermal capacitance";',
'  parameter Real c_p=1.2 "Specific heat";',
'  Real T "Temperature";',
'initial equation',
'  T = T0 "Specify initial value for T";',
'equation',
'  m*c_p*der(T) = h*A*(T_inf-T) "Newton s law of cooling";',
'end NewtonCooling;"'
))

In [7]:
# check this input file is well read by plugin
ParseInput(model="Modelica",input.files="NewtonCooling.mo.par")

$convection
[1] NA

In [8]:
print(Grid())

     Use       Computer   host.name                OS      address.port
1  true   1620873a26fc   localhost   Linux 5.10.133+   127.0.0.1:41649 
  local.status      since activity         codes
1        free   19:32:30     idle   R, Modelica 


In [9]:
# launch 4 calculations for different `convection` values (0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
Run(model="Modelica",input.files="NewtonCooling.mo.par", input.variables=list(convection=c(0.5,0.6,0.7,0.8,0.9,1.0)), output.expressions="min(T)")

Using archive directory: /content
 Running:	0/6                                                                    Interrupt !
Interrupt !


ERROR: ignored

In [ ]:
# install Brent root finding algorithm
install.Design('Brent')

In [ ]:
# find the `convection` value leading to `min(T) = 25.2` (with relative precision of 0.01 on `convection` value)
RunDesign(model="Modelica",input.files="NewtonCooling.mo.par", input.variables=list(convection="[0.5,1.0]"), output.expressions="min(T)", 
          design="Brent", design_options=list(ytarget=25.2, ytol=0.01))